# Tomographic reconstrution

The goal of this assignment is to implement a tomographic reconstruction, first in 2D and then in 3D.
You can use numpy and any library you want to open the images, but no library that do all or part of the reconstruction.

This notebook come with two folders : <br>
&nbsp;&nbsp;&nbsp;./&nbsp;+<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|--AR_Assignment3.ipynb<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|--data/\*<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|--notebook_images/\*<br>
 


## I) Parallel 2D reconstruction (3pts)

The first step will be to reconstruct a 2D image from 1D projections.

The reconstructions algorithms are based on two function : the projection and the backprojection.

The projection is the process done when capturing a radiograph. It reduce the dimension of an object by one (2D->1D or 3D->2D). 

The backprojection is the inverse operation of the projection : it take a projected data and propagate it into a higher dimension object (1D->2D, 2D->3D). Of course, a lot of information is lost in the projection process, so the result of the backprojection carry a lot less information than the original object. In order to obtain a better approximation, a lot of projections at different positions are needed.

For both the projection and the backprojection, you need to link which voxel is linked with which pixels. From a backprojection perspective, it's better to start at each pixel and determine which voxels are projected to this pixel. It can be done by tracing a discrete path in the volume, with the start and end of this path depending on the geometry of the aquisition.

So, you need to compute for each pixels of the projected data the path in the reconstructed object between the pixel and the source, and add to all element in this path the value of the pixel divided by the length of the path.

For an improved image quality, the path can be also weighted (anti-aliased) to produce a smoother result. In this assignment, you are not asked to do that and can consider all elements in the path to have a weigth of 1.

<img src="notebook_images/projection_2d_par.jpg" alt="Projection 2D parallel" style="height: 20em;"/>

In the case of a parallel geometry, the voxels have the same size as the pixels, and the distances between the source, the object and the detector don't impact the result.

**Step 1.0 :** Write a function that take two 2D points in a grid as argument and return the list of all the cells (coordinates) that are in the path between them.

In [22]:
def get_cells(p1, p2):
    # Setup initial conditions
    x1, y1 = p1
    x2, y2 = p2
    dx = x2 - x1
    dy = y2 - y1
    # Determine how steep the line is
    is_steep = abs(dy) > abs(dx)

    # Rotate line
    if is_steep:
        x1, y1 = y1, x1
        x2, y2 = y2, x2

    # Swap start and end points if necessary and store swap state
    swapped = False
    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1
        swapped = True

    # Recalculate differentials
    dx = x2 - x1
    dy = y2 - y1

    # Calculate error
    error = int(dx / 2.0)
    ystep = 1 if y1 < y2 else -1

    # Iterate over bounding box generating points between start and end
    y = y1
    points = []
    for x in range(x1, x2 + 1):
        coord = (y, x) if is_steep else (x, y)
        points.append(coord)
        error -= abs(dy)
        if error < 0:
            y += ystep
            error += dx
    # Reverse the list if the coordinates were swapped
    if swapped:
        points.reverse()
    return points

**Step 1.1 :** Create a numpy array of size 80 by 80 filled with zeros. Using the previous function, fill the path from \[3,5\] to \[70,75\] with ones and display the result  with gnuplot.

In [23]:
import numpy as np
arr=np.zeros([80,80])
pathCor=get_cells((3,5),(70,75))
for point in pathCor:
    arr[point]=1


**Step 1.2 :** Write a function that take as argument a coordinate on a sinogram line and the angle of this line (and possibly other parameters to represent the geometry) and return the start and end of the path between this pixel and the source in a parallel geometry.

In [24]:
def draw_line(point,angle,xlim = 10, ylim = 10):
    x,y = point
    
    if angle%180==0:
        m = 0
    elif angle==90 and angle==270:
        m = 2
    else:
        #slope
        m = math.tan(math.radians(angle))
    #find the y intercept of y=mx+c
    c = y - m*x
    if abs(m)<=1:
        x2 = xlim
        point_end = [x2,np.round(m*x2 +c)]
        point_start = [-x2,np.round(-m*x2 +c)]
    else:
        y2 = ylim
        point_end = [np.round((y2-c)/m),y2]
        point_start = [np.round((-y2-c)/m),-y2]
    return point_start,point_end

In [25]:
p = [-13,-13]
angle = 180
[p1,p2] = draw_line(p,angle)
print(p1,p2)

[-10, -13] [10, -13]


**Step 1.3 :** Create a function that take as argument a sinogram and the needed parameters, and output the result of the 2D backprojection. You should use the previous functions.

**Step 1.4 :** Open the tiff file "data/sinogram_2d_parallel.tif" and display it. Each line of the sinogram represent the projection of the image at a given angle.

The parameters of this projection are the following :
 - Parallel aqcuisition geometry
 - 200 projections over 360 degrees (with a regular step)

**Step 1.5 :** Apply the parallel reconstruction function to the parallel sinogram and display the result

## 2) Spherical 2D reconstruction (3pts)

The parallel geometry is simpler to reconstruct, but in practice not used so often because of the complexity of the parallel x-ray generation. A much more common approach is to use a ponctual source, that create a spherical geometry. 

The path come from the source and go to a pixel on the detector. In 2D, it can be simplifies as a fan-beam when the source and the detector are aligned.

<img src="notebook_images/projection_2d_cone.jpg" alt="Projection 2D spherical" style="height: 20em;"/>

**Step 2.0 :** Write a function that take as argument a coordinate on a sinogram line and the angle of this line (and possibly other parameters to represent the geometry) and return the start and end of the path between this pixel and the source in a spherical geometry.

**Step 2.1 :** Create a function that take as argument a sinogram and the needed parameters, and output the result of the 2D spherical back-projection. You should use the previous functions.

**Step 2.2 :** Open the tiff file "data/sinogram_2d_spherical.tif" and display it.

The parameters of the projection are the following :
 - Spherical aqcuisition geometry
 - 200 projections over 360 degrees
 - 300mm
 - 400mm
 - Voxel size of 1mm

**Step 2.3 :** Apply the spherical reconstruction function to the spherical sinogram and display the result

## 3) Spherical 3D reconstruction (3pts + 1pt for performances)

Now we are going to extend this algorithm to 3D.

Going from 2D to 3D means that a lot more computations have to be done. 
Using only python for-loop for the 3D reconstruction will be very slow. It's recommended to use as much as possible the numpy functions to avoid having to use too many for-loop.

<img src="notebook_images/projection_3d_cone.jpg" alt="Projection 3D spherical" style="height: 20em;"/>

 **Step 3.0 :** Write a function that return the coordinates of all voxels between two points in a 3D grid
 

**Step 3.1 :** Write a function that take as argument a coordinate on a projection and an angle (and possibly other parameters to represent the geometry) and return the start and end of the path between this pixel and the source in a 3D spherical geometry.

**Step 3.2 :** Create a function that take as argument a list of projection and the needed parameters, and output the result of the 3D spherical back-projection.

**Step 3.3 : ** Open all the tiff files in "data/3d_projections/"

The parameters of this dataset are the following :

 - Spherical aqcuisition geometry
 - The source, the rotation axis and the detector are all aligned
 - 200 projections over 360 degrees
 - Distance source-object : 200mm
 - Distance source-detector : 2000mm
 - Pixel size : 0.5mm by 0.5mm

**Step 3.4 :** Apply the function to the dataset and display the result of an arbitrary layer

In [26]:
DISPLAYED_LAYER_INDEX = 64

...

Ellipsis